# Project K
Korean pop culture has grown strongly in and outside Asia. The music scene has seen an explosion of commercially successful artists and I find it quite hard to keep up with the latest trends. In this project, I experiment with two free API services - **Last.FM** and **Twitter**. I build a dataset containing top k-pop albums linked to the artists' official twitter accounts and use the data to feed a monitoring dashboard built in Tableau.

In the dashboard, I want to see album data, artist latest tweets, and some basic visualizations showing metrics such as track playcount, Twitter followers, likes, retweets and etc. The objective of this project is to get familiar with how APIs work, get some practice with pandas data wrangling, and test Tableau Public. 

In [1]:
# Start by importing libraries
import requests
import json
import pandas as pd
import time

# Part A - Last.FM API
### 1) Extract Last.FM data
Last.FM is a music streaming website that has a great public API available for use. After quickly reading the documentation and performing some tests, I decided to use Python's requests library to pass arguments and get data. All important argumnents are passed in payload_album. Notice that I am using `'method': 'getTopAlbums'` and `'tag': 'kpop'` to get top albums tagged as k-pop.

In [2]:
# LastFM API

# API_KEY and API_SECRET
API_KEY = "b3e2c8ae896f92285eba81fcfac559dd"
API_SECRET = "73b32d6ab2954d5088e5043e479ba0e6"

# Set headers and payload
headers = {'user-agent': 'ochbruno'}
payload_album = {'api_key': API_KEY, 'method': 'tag.getTopAlbums', 'tag':'kpop', 'format': 'json'}

# Get top kpop albuns
top_albums_r = requests.get('https://ws.audioscrobbler.com/2.0/', headers=headers, params=payload_album)

# Check URL of the request
top_albums_r

<Response [200]>

API returned **Response 200**, which means my request was sucessful. Now, I want check the data and see how it is structured. 

One of the argumets in the payload was `'format':'json'`, which means that I asked the API to return the data in JSON format (default was in XML). Now, I need to use the json library to be able to navigate the data in the same way I would do with Python dictionaries and lists.  

In [3]:
# Data output: check first entry
top_albums_r.json()['albums']['album'][0]

{'name': 'Dynamite',
 'mbid': '',
 'url': 'https://www.last.fm/music/BTS',
 'artist': {'name': 'BTS',
  'mbid': '0d79fe8e-ba27-4859-bb8c-2f255f346853',
  'url': 'https://www.last.fm/music/BTS'},
 'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/41b15d8a0ad6a81323b598bfb19cede9.png',
   'size': 'small'},
  {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/41b15d8a0ad6a81323b598bfb19cede9.png',
   'size': 'medium'},
  {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/41b15d8a0ad6a81323b598bfb19cede9.png',
   'size': 'large'},
  {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/41b15d8a0ad6a81323b598bfb19cede9.png',
   'size': 'extralarge'}],
 '@attr': {'rank': '1'}}

### 2) Get top albums data
Store the data extracted above in a dataframe. I am turning all artist names into uppercase to avoid problems when I merge data in the later steps of this project. 

In [4]:
# For loop to store top kpop albums in a dataframme
top_albums_json = top_albums_r.json()['albums']['album']

d={}
d["artist"]=[]
d["album"]=[]
d["rank"]=[]
for i, val in enumerate(top_albums_json):
    d["artist"].append(val["artist"]["name"])
    d["album"].append(val["name"])
    d["rank"].append(val["@attr"]["rank"])

top_albums = pd.DataFrame(d).head(20)

#create a column with artist name in uppercase
top_albums["artist"] = top_albums["artist"].apply(lambda x: x.upper())

top_albums

,artist,album,rank
0,BTS,Dynamite,1
1,TOMORROW X TOGETHER,The Chaos Chapter: FREEZE,2
2,J-HOPE,Hope World,3
3,BTS,Dark & Wild,4
4,LISA,LALISA,5
5,TOMORROW X TOGETHER,The Dream Chapter: MAGIC,6
6,BLACKPINK,SQUARE UP,7
7,STRAY KIDS,NOEASY,8
8,LOONA,[12:00],9
9,ROSÉ,R,10


I am using only the top 20 albums to keep the project simple. 

As of December 2021, I can see that a couple of artists have multiple albums performing at the top. Doing a little bit of research, I also learned that some of the top albums were solo projects of the these bands members. For example, BTS has 4 entries in the top 20 and J-Hope is a member of BTS. Same with Blackpink and Rose/Lisa. It seems that these two groups are quite strong in the k-pop scene.

### 3) Get album info
Below, I set the parameters to get further album information (basically, playcount, listeners, and track names). I build a for loop that will go line by line and feed artist and album name into the `'method': 'album.getInfo'`.

In [5]:
# Do a for loop to get album info for each album in top_albums
albuminfo_list = []
for artist, album in zip(top_albums["artist"], top_albums["album"]):
    payload_albuminfo = {'api_key': API_KEY, 'method': 'album.getInfo', 'artist':artist, 'album':album,'format': 'json'}
    albuminfo_list.append(requests.get('https://ws.audioscrobbler.com/2.0/', headers=headers, params=payload_albuminfo).json())
    time.sleep(6)

Let's check first entry and prepare a loop to store relevant data in a dataframe

In [6]:
# Data output: check first entry
albuminfo_list[0]["album"]

{'artist': 'BTS',
 'mbid': '0da3ed2f-392f-4c80-848f-193f38b0d0b3',
 'tags': {'tag': [{'url': 'https://www.last.fm/tag/pop', 'name': 'pop'},
   {'url': 'https://www.last.fm/tag/bts', 'name': 'bts'},
   {'url': 'https://www.last.fm/tag/disco', 'name': 'disco'},
   {'url': 'https://www.last.fm/tag/kpop', 'name': 'kpop'},
   {'url': 'https://www.last.fm/tag/funk', 'name': 'funk'}]},
 'playcount': '28673011',
 'image': [{'size': 'small',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/34s/41b15d8a0ad6a81323b598bfb19cede9.jpg'},
  {'size': 'medium',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/64s/41b15d8a0ad6a81323b598bfb19cede9.jpg'},
  {'size': 'large',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/174s/41b15d8a0ad6a81323b598bfb19cede9.jpg'},
  {'size': 'extralarge',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/41b15d8a0ad6a81323b598bfb19cede9.jpg'},
  {'size': 'mega',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/41b15d8a0ad6a81323b598bfb19cede9.j

In [7]:
# For loop to store the data into a dataframe
x = {}
x["artist"]=[]
x["name"]=[]
x["playcount"]=[]
x["listeners"]=[]
x["published"]=[]
x["jpg"]=[]
x["numtracks"]=[]
x["trackname"]=[]
x["tracksdur"]=[]

for album in albuminfo_list:
    x["artist"].append(album["album"]["artist"])
    x["name"].append(album["album"]["name"])
    x["playcount"].append(album["album"]["playcount"])
    x["listeners"].append(album["album"]["listeners"])
    x["jpg"].append(album["album"]["image"][2]["#text"])
    try:
        x["published"].append(album["album"]["wiki"]["published"])
    except:
        x["published"].append("NA")
    try:
        x["numtracks"].append(len(album["album"]["tracks"]["track"]))
    except:
        x["numtracks"].append(0)
    try:
        x["trackname"].append([t["name"] for t in album["album"]["tracks"]["track"]])
    except:
        x["trackname"].append(["NA"])
    try:
        x["tracksdur"].append([t["duration"] for t in album["album"]["tracks"]["track"]])
    except:
        x["tracksdur"].append(["0"])

# Store track data into a dataframe and bring rank column from top_albums 
albuminfo = pd.DataFrame(x)
albuminfo = albuminfo.merge(right=top_albums["rank"], left_index=True, right_index=True)

In [8]:
# Check albuminfo data
albuminfo

,artist,name,playcount,listeners,published,jpg,numtracks,trackname,tracksdur,rank
0,BTS,Dynamite,28673011,248325,"11 Nov 2021, 19:26",https://lastfm.freetls.fastly.net/i/u/174s/41b...,2,"[Dynamite, Dynamite - Instrumental]","[199, 198]",1
1,TOMORROW X TOGETHER,The Chaos Chapter: FREEZE,24279001,166105,"11 Nov 2021, 13:17",https://lastfm.freetls.fastly.net/i/u/174s/d5e...,8,"[ANTI-ROMANTIC, 0X1=LOVESONG (I Know I Love Yo...","[215, 202, 160, 203, 186, 186, 195, 195]",2
2,j-hope,Hope World,23430617,195267,"09 Sep 2021, 19:49",https://lastfm.freetls.fastly.net/i/u/174s/57b...,7,"[Hope World, P.O.P (Piece Of Peace) pt.1, 백일몽 ...","[204, 181, 228, 89, 229, 197, 90]",3
3,BTS,DARK & WILD,16051931,200195,"10 Oct 2021, 21:08",https://lastfm.freetls.fastly.net/i/u/174s/67f...,14,"[Intro: What Am I to You, Danger, War of Hormo...","[165, 245, 266, 257, 254, 265, None, 41, 234, ...",4
4,Lisa,LALISA,10477044,230773,"09 Sep 2021, 18:54",https://lastfm.freetls.fastly.net/i/u/174s/386...,2,"[LALISA, MONEY]","[200, 168]",5
5,TOMORROW X TOGETHER,The Dream Chapter: MAGIC,10802108,160810,"11 Nov 2019, 20:29",https://lastfm.freetls.fastly.net/i/u/174s/ac8...,8,"[New Rules, 9 and Three Quarters (Run Away), R...","[175, 211, 214, 193, 230, 193, 217, 232]",6
6,BLACKPINK,SQUARE UP,10644536,292527,"07 Jul 2021, 16:47",https://lastfm.freetls.fastly.net/i/u/174s/f18...,5,"[뚜두뚜두 (DDU-DU DDU-DU), Forever Young, REALLY, ...","[215, 237, 197, 216, 213]",7
7,Stray Kids,NOEASY,9516958,93995,"12 Dec 2021, 21:38",https://lastfm.freetls.fastly.net/i/u/174s/8f3...,14,"[CHEESE, Thunderous, DOMINO, Ssick, The View, ...","[182, 183, 198, 190, 201, 177, 209, 209, 215, ...",8
8,Loona,[12:00],8706683,149945,"03 Mar 2021, 15:41",https://lastfm.freetls.fastly.net/i/u/174s/148...,8,"[12:00, Why Not?, Voice, Fall Again, Universe,...","[73, 205, 198, 215, 214, 182, 162, 198]",9
9,Rosé,R,8510730,178075,"09 Sep 2021, 16:35",https://lastfm.freetls.fastly.net/i/u/174s/3e4...,0,[NA],[0],10


Data above shows all k-pop albums with columns containing track names and track duration. Before proceeding to the next step, I will save the jpg files in my work folder. These are album jackets for each top album and I will use then in the dashboard.

In [9]:
# loop through every jpg row and save the jpg file
# I use generic file names so that it is easy to update when it is linked to the dashboard
import urllib.request

for idx, url in enumerate(albuminfo["jpg"]):
    filename = albuminfo.loc[idx,"rank"].zfill(2)+"_albumrank"+url[-4:]
    urllib.request.urlretrieve(url, filename)
    print("saved "+filename)

saved 01_albumrank.jpg
saved 02_albumrank.jpg
saved 03_albumrank.jpg
saved 04_albumrank.png
saved 05_albumrank.jpg
saved 06_albumrank.jpg
saved 07_albumrank.jpg
saved 08_albumrank.png
saved 09_albumrank.png
saved 10_albumrank.jpg
saved 11_albumrank.jpg
saved 12_albumrank.jpg
saved 13_albumrank.jpg
saved 14_albumrank.jpg
saved 15_albumrank.jpg
saved 16_albumrank.png
saved 17_albumrank.png
saved 18_albumrank.jpg
saved 19_albumrank.jpg
saved 20_albumrank.png


### 4) Process track names and duration
Note how `trackname` and `tracksdur` are actually lists inside rows. As the name suggests, `trackname`  is the name of track and `tracksdur` is its duration in seconds. Each album has multiple tracks and these need to be transposed so that one track fits inside a single row. In the next couple of cells, I will perform this task by transposing both variables and merging those back to albuminfo.

In [10]:
# transform trackname column into a list and put it back in a dataframe.
# note how now I have a dataframe where each element populates a cell.
tracknametbl = pd.DataFrame(albuminfo["trackname"].tolist())
tracknametbl

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,Dynamite,Dynamite - Instrumental,None,None,None,None,None,None,None,None,None,None,None,None
1,ANTI-ROMANTIC,0X1=LOVESONG (I Know I Love You) (feat. Seori),MAGIC,소악행,밸런스 게임),No Rules,디어 스푸트니크,Frost,None,None,None,None,None,None
2,Hope World,P.O.P (Piece Of Peace) pt.1,백일몽 (Daydream),Base Line,항상 (HANGSANG),Airplane,Blue Side (Outro),None,None,None,None,None,None,None
3,Intro: What Am I to You,Danger,War of Hormone,Hip Hop Phile,Let Me Know,Rain,BTS Cypher PT.3 : Killer [feat. Supreme Boi],Interlude : What Are You Doing Now,Could You Turn Off Your Cell Phone,Embarrassed,24/7=heaven,Look Here,So 4 more,Outro : Do You Think It Makes Sense?
4,LALISA,MONEY,None,None,None,None,None,None,None,None,None,None,None,None
5,New Rules,9 and Three Quarters (Run Away),Roller Coaster,Poppin' Star,Can't We Just Leave The Monster Alive?,Magic Island,20cm,Angel Or Devil,None,None,None,None,None,None
6,뚜두뚜두 (DDU-DU DDU-DU),Forever Young,REALLY,SEE U LATER,마지막처럼,None,None,None,None,None,None,None,None,None
7,CHEESE,Thunderous,DOMINO,Ssick,The View,"Sorry, I Love You",Silent Cry,Secret Secret,Star Lost,"Red Lights (Bang Chan, Hyunjin)","Surfin' (Lee Know, Changbin, Felix)","Gone Away (Han, Seungmin, I.N)",WOLFGANG,Mixtape : OH
8,12:00,Why Not?,Voice,Fall Again,Universe,Hide & Seek,OOPS!,Star,None,None,None,None,None,None
9,NA,None,None,None,None,None,None,None,None,None,None,None,None,None


In [11]:
# build a temporary dataframe "t" and merge that with dataframe of track names 
t = albuminfo.drop(["trackname", "tracksdur"], axis=1)
albuminfo_1 = t.merge(right=tracknametbl, how='left', left_index=True, right_index=True)

# use melt to unpivot track names
albuminfo_1 = albuminfo_1.melt(id_vars=t.columns, value_name='trackname_melt')

# check result
albuminfo_1

,artist,name,playcount,listeners,published,jpg,numtracks,rank,variable,trackname_melt
0,BTS,Dynamite,28673011,248325,"11 Nov 2021, 19:26",https://lastfm.freetls.fastly.net/i/u/174s/41b...,2,1,0,Dynamite
1,TOMORROW X TOGETHER,The Chaos Chapter: FREEZE,24279001,166105,"11 Nov 2021, 13:17",https://lastfm.freetls.fastly.net/i/u/174s/d5e...,8,2,0,ANTI-ROMANTIC
2,j-hope,Hope World,23430617,195267,"09 Sep 2021, 19:49",https://lastfm.freetls.fastly.net/i/u/174s/57b...,7,3,0,Hope World
3,BTS,DARK & WILD,16051931,200195,"10 Oct 2021, 21:08",https://lastfm.freetls.fastly.net/i/u/174s/67f...,14,4,0,Intro: What Am I to You
4,Lisa,LALISA,10477044,230773,"09 Sep 2021, 18:54",https://lastfm.freetls.fastly.net/i/u/174s/386...,2,5,0,LALISA
...,...,...,...,...,...,...,...,...,...,...
275,BLACKPINK,SQUARE ONE,6143917,253888,"12 Dec 2020, 21:48",https://lastfm.freetls.fastly.net/i/u/174s/fad...,2,16,13,None
276,ITZY,GUESS WHO,5454995,144705,"05 May 2021, 22:22",https://lastfm.freetls.fastly.net/i/u/174s/95a...,6,17,13,None
277,IZ*ONE,BLOOM*IZ,5326176,92662,"02 Feb 2020, 14:43",https://lastfm.freetls.fastly.net/i/u/174s/c76...,6,18,13,None
278,K/DA,ALL OUT,5115044,140332,"01 Jan 2021, 15:45",https://lastfm.freetls.fastly.net/i/u/174s/335...,5,19,13,None


Now all track names have its own row and duplicates were created for all data associated with album data. See the case for this group called "ITZY": 

In [12]:
albuminfo_1[albuminfo_1["artist"]=="ITZY"]

,artist,name,playcount,listeners,published,jpg,numtracks,rank,variable,trackname_melt
16,ITZY,GUESS WHO,5454995,144705,"05 May 2021, 22:22",https://lastfm.freetls.fastly.net/i/u/174s/95a...,6,17,0,In the morning
36,ITZY,GUESS WHO,5454995,144705,"05 May 2021, 22:22",https://lastfm.freetls.fastly.net/i/u/174s/95a...,6,17,1,Sorry Not Sorry
56,ITZY,GUESS WHO,5454995,144705,"05 May 2021, 22:22",https://lastfm.freetls.fastly.net/i/u/174s/95a...,6,17,2,KIDDING ME
76,ITZY,GUESS WHO,5454995,144705,"05 May 2021, 22:22",https://lastfm.freetls.fastly.net/i/u/174s/95a...,6,17,3,Wild Wild West
96,ITZY,GUESS WHO,5454995,144705,"05 May 2021, 22:22",https://lastfm.freetls.fastly.net/i/u/174s/95a...,6,17,4,SHOOT!
116,ITZY,GUESS WHO,5454995,144705,"05 May 2021, 22:22",https://lastfm.freetls.fastly.net/i/u/174s/95a...,6,17,5,TENNIS (0:0)
136,ITZY,GUESS WHO,5454995,144705,"05 May 2021, 22:22",https://lastfm.freetls.fastly.net/i/u/174s/95a...,6,17,6,None
156,ITZY,GUESS WHO,5454995,144705,"05 May 2021, 22:22",https://lastfm.freetls.fastly.net/i/u/174s/95a...,6,17,7,None
176,ITZY,GUESS WHO,5454995,144705,"05 May 2021, 22:22",https://lastfm.freetls.fastly.net/i/u/174s/95a...,6,17,8,None
196,ITZY,GUESS WHO,5454995,144705,"05 May 2021, 22:22",https://lastfm.freetls.fastly.net/i/u/174s/95a...,6,17,9,None


There are several rows with None and that was totally expected. I will remove those in later steps.

Below, I do exactly the same thing for `tracksdur` and merge the results to get a single table containing track names and track duration.

In [13]:
# I do exact the same procedure for tracksdur
# will skip commentary
tracksdurtbl = pd.DataFrame(albuminfo["tracksdur"].tolist())
albuminfo_2 = t.merge(right=tracksdurtbl, how='left', left_index=True, right_index=True)
albuminfo_2 = albuminfo_2.melt(id_vars=t.columns, value_name='tracksdur_melt')
albuminfo_2

# now let's bring unpivoted tracksdur to the table with unpivoted track names.
albuminfo_3 = albuminfo_1.merge(albuminfo_2["tracksdur_melt"], left_index=True, right_index=True)
albuminfo_3

,artist,name,playcount,listeners,published,jpg,numtracks,rank,variable,trackname_melt,tracksdur_melt
0,BTS,Dynamite,28673011,248325,"11 Nov 2021, 19:26",https://lastfm.freetls.fastly.net/i/u/174s/41b...,2,1,0,Dynamite,199
1,TOMORROW X TOGETHER,The Chaos Chapter: FREEZE,24279001,166105,"11 Nov 2021, 13:17",https://lastfm.freetls.fastly.net/i/u/174s/d5e...,8,2,0,ANTI-ROMANTIC,215
2,j-hope,Hope World,23430617,195267,"09 Sep 2021, 19:49",https://lastfm.freetls.fastly.net/i/u/174s/57b...,7,3,0,Hope World,204
3,BTS,DARK & WILD,16051931,200195,"10 Oct 2021, 21:08",https://lastfm.freetls.fastly.net/i/u/174s/67f...,14,4,0,Intro: What Am I to You,165
4,Lisa,LALISA,10477044,230773,"09 Sep 2021, 18:54",https://lastfm.freetls.fastly.net/i/u/174s/386...,2,5,0,LALISA,200
...,...,...,...,...,...,...,...,...,...,...,...
275,BLACKPINK,SQUARE ONE,6143917,253888,"12 Dec 2020, 21:48",https://lastfm.freetls.fastly.net/i/u/174s/fad...,2,16,13,None,NaN
276,ITZY,GUESS WHO,5454995,144705,"05 May 2021, 22:22",https://lastfm.freetls.fastly.net/i/u/174s/95a...,6,17,13,None,NaN
277,IZ*ONE,BLOOM*IZ,5326176,92662,"02 Feb 2020, 14:43",https://lastfm.freetls.fastly.net/i/u/174s/c76...,6,18,13,None,NaN
278,K/DA,ALL OUT,5115044,140332,"01 Jan 2021, 15:45",https://lastfm.freetls.fastly.net/i/u/174s/335...,5,19,13,None,NaN


Now I proceed to do some renaming and remove null rows.

In [14]:
# rename "variable" to trackno (tracknumber) and add 1
albuminfo_melt = albuminfo_3.rename(columns={"variable":"trackno"})
albuminfo_melt["trackno"]=albuminfo_melt["trackno"]+1
albuminfo_melt = albuminfo_melt.dropna()

# show the data
albuminfo_melt = albuminfo_melt.reset_index(drop=True)
albuminfo_melt

,artist,name,playcount,listeners,published,jpg,numtracks,rank,trackno,trackname_melt,tracksdur_melt
0,BTS,Dynamite,28673011,248325,"11 Nov 2021, 19:26",https://lastfm.freetls.fastly.net/i/u/174s/41b...,2,1,1,Dynamite,199
1,TOMORROW X TOGETHER,The Chaos Chapter: FREEZE,24279001,166105,"11 Nov 2021, 13:17",https://lastfm.freetls.fastly.net/i/u/174s/d5e...,8,2,1,ANTI-ROMANTIC,215
2,j-hope,Hope World,23430617,195267,"09 Sep 2021, 19:49",https://lastfm.freetls.fastly.net/i/u/174s/57b...,7,3,1,Hope World,204
3,BTS,DARK & WILD,16051931,200195,"10 Oct 2021, 21:08",https://lastfm.freetls.fastly.net/i/u/174s/67f...,14,4,1,Intro: What Am I to You,165
4,Lisa,LALISA,10477044,230773,"09 Sep 2021, 18:54",https://lastfm.freetls.fastly.net/i/u/174s/386...,2,5,1,LALISA,200
...,...,...,...,...,...,...,...,...,...,...,...
109,Stray Kids,NOEASY,9516958,93995,"12 Dec 2021, 21:38",https://lastfm.freetls.fastly.net/i/u/174s/8f3...,14,8,13,WOLFGANG,192.0
110,NCT,NCT 2018 EMPATHY,6431069,149895,"03 Mar 2018, 02:22",https://lastfm.freetls.fastly.net/i/u/174s/a84...,14,15,13,OUTRO: VISION,57.0
111,BTS,DARK & WILD,16051931,200195,"10 Oct 2021, 21:08",https://lastfm.freetls.fastly.net/i/u/174s/67f...,14,4,14,Outro : Do You Think It Makes Sense?,172.0
112,Stray Kids,NOEASY,9516958,93995,"12 Dec 2021, 21:38",https://lastfm.freetls.fastly.net/i/u/174s/8f3...,14,8,14,Mixtape : OH,212.0


### 5) Get more track data (= track playcount)
The data is already looking quite good, but it is missing one important variable: **track playcount**. The playcount data that I currently have is **album playcount** which is not good enough if I want to analyze track as well. LastFM's `'method': 'album.getInfo'` does not store a lot of track information, so I need to set a new request using `'method':'track.getInfo'`. 

The code is quite similar to what I have already written above. So I will skip commentary and get to the final data to avoid being too repetitive.  

In [15]:
from tqdm.notebook import tqdm
bar = tqdm(total = albuminfo_melt.shape[0])

trackinfo_list = []
for artist, track in zip(albuminfo_melt["artist"], albuminfo_melt["trackname_melt"]):
    payload_trackinfo = {'api_key': API_KEY, 'method': 'track.getInfo', 'artist':artist, 'track':track,'format': 'json'}
    trackinfo_list.append(requests.get('https://ws.audioscrobbler.com/2.0/', headers=headers, params=payload_trackinfo).json())
    bar.update(1)
    time.sleep(3)

  0%|          | 0/114 [00:00<?, ?it/s]

In [16]:
trackinfo_list[0]["track"]

{'name': 'Dynamite',
 'url': 'https://www.last.fm/music/BTS/_/Dynamite',
 'duration': '199000',
 'streamable': {'#text': '0', 'fulltrack': '0'},
 'listeners': '435982',
 'playcount': '80642441',
 'artist': {'name': 'BTS',
  'mbid': '0d79fe8e-ba27-4859-bb8c-2f255f346853',
  'url': 'https://www.last.fm/music/BTS'},
 'album': {'artist': 'BTS',
  'title': 'Dynamite (Extended)',
  'url': 'https://www.last.fm/music/BTS/Dynamite+(Extended)',
  'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/0e41a35afb8e2ad81aca9621d420a33f.png',
    'size': 'small'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/0e41a35afb8e2ad81aca9621d420a33f.png',
    'size': 'medium'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/0e41a35afb8e2ad81aca9621d420a33f.png',
    'size': 'large'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/0e41a35afb8e2ad81aca9621d420a33f.png',
    'size': 'extralarge'}]},
 'toptags': {'tag': [{'name': 'pop', 'url': 'https://www.last.fm/tag/po

In [17]:
name=[]
album=[]
artist=[]
playcount=[]
listeners=[]

for t in range(len(trackinfo_list)):
    try:
        name.append(trackinfo_list[t]["track"]["name"])
    except:
        name.append("NA")        
    try:
        album.append(trackinfo_list[t]["track"]["album"]["title"])
    except:
        album.append("NA")
    try:
        artist.append(trackinfo_list[t]["track"]["artist"]["name"])
    except:
        artist.append("NA")
    try:
        playcount.append(trackinfo_list[t]["track"]["playcount"])
    except:
        playcount.append(0)
    try:
        listeners.append(trackinfo_list[t]["track"]["listeners"])
    except:
        listeners.append(0)

trackinfo_df = pd.DataFrame(dict(name=name,album=album, artist=artist, playcount=playcount, listeners=listeners))
trackinfo_df

,name,album,artist,playcount,listeners
0,Dynamite,Dynamite (Extended),BTS,80642441,435982
1,ANTI-ROMANTIC,The Chaos Chapter: FREEZE,TOMORROW X TOGETHER,3867105,136965
2,Hope World,NA,J-Hope,3106115,118107
3,Intro: What Am I to You,DARK & WILD,BTS,776556,66698
4,LALISA,LALISA,Lisa,5342514,138270
...,...,...,...,...,...
109,WOLFGANG,NA,STRAY KIDS,962894,40779
110,OUTRO: VISION,NCT 2018 EMPATHY,NCT,95244,13050
111,Outro : Do You Think It Makes Sense?,DARK&WILD,BTS,29134,3661
112,Mixtape : OH,NA,STRAY KIDS,1092738,51049


In [18]:
albuminfo_data = albuminfo_melt.merge(right=trackinfo_df, left_index=True, right_index=True)
albuminfo_data.head(20)

,artist_x,name_x,playcount_x,listeners_x,published,jpg,numtracks,rank,trackno,trackname_melt,tracksdur_melt,name_y,album,artist_y,playcount_y,listeners_y
0,BTS,Dynamite,28673011,248325,"11 Nov 2021, 19:26",https://lastfm.freetls.fastly.net/i/u/174s/41b...,2,1,1,Dynamite,199,Dynamite,Dynamite (Extended),BTS,80642441,435982
1,TOMORROW X TOGETHER,The Chaos Chapter: FREEZE,24279001,166105,"11 Nov 2021, 13:17",https://lastfm.freetls.fastly.net/i/u/174s/d5e...,8,2,1,ANTI-ROMANTIC,215,ANTI-ROMANTIC,The Chaos Chapter: FREEZE,TOMORROW X TOGETHER,3867105,136965
2,j-hope,Hope World,23430617,195267,"09 Sep 2021, 19:49",https://lastfm.freetls.fastly.net/i/u/174s/57b...,7,3,1,Hope World,204,Hope World,NA,J-Hope,3106115,118107
3,BTS,DARK & WILD,16051931,200195,"10 Oct 2021, 21:08",https://lastfm.freetls.fastly.net/i/u/174s/67f...,14,4,1,Intro: What Am I to You,165,Intro: What Am I to You,DARK & WILD,BTS,776556,66698
4,Lisa,LALISA,10477044,230773,"09 Sep 2021, 18:54",https://lastfm.freetls.fastly.net/i/u/174s/386...,2,5,1,LALISA,200,LALISA,LALISA,Lisa,5342514,138270
5,TOMORROW X TOGETHER,The Dream Chapter: MAGIC,10802108,160810,"11 Nov 2019, 20:29",https://lastfm.freetls.fastly.net/i/u/174s/ac8...,8,6,1,New Rules,175,New Rules,The Dream Chapter: MAGIC,TOMORROW X TOGETHER,1422960,83220
6,BLACKPINK,SQUARE UP,10644536,292527,"07 Jul 2021, 16:47",https://lastfm.freetls.fastly.net/i/u/174s/f18...,5,7,1,뚜두뚜두 (DDU-DU DDU-DU),215,뚜두뚜두 (DDU-DU DDU-DU),BLACKPINK 2018 Tour In Your Area Seoul,BLACKPINK,190195,8230
7,Stray Kids,NOEASY,9516958,93995,"12 Dec 2021, 21:38",https://lastfm.freetls.fastly.net/i/u/174s/8f3...,14,8,1,CHEESE,182,CHEESE,NA,STRAY KIDS,699574,41359
8,Loona,[12:00],8706683,149945,"03 Mar 2021, 15:41",https://lastfm.freetls.fastly.net/i/u/174s/148...,8,9,1,12:00,73,12:00,[12:00],Loona,459900,43076
9,Rosé,R,8510730,178075,"09 Sep 2021, 16:35",https://lastfm.freetls.fastly.net/i/u/174s/3e4...,0,10,1,NA,0,NA,NA,NA,0,0


In [19]:
albuminfo_data = albuminfo_data.drop(["name_y", "album", "artist_y"], axis=1)

Now I have playcount and listener data!

In [20]:
albuminfo_data

,artist_x,name_x,playcount_x,listeners_x,published,jpg,numtracks,rank,trackno,trackname_melt,tracksdur_melt,playcount_y,listeners_y
0,BTS,Dynamite,28673011,248325,"11 Nov 2021, 19:26",https://lastfm.freetls.fastly.net/i/u/174s/41b...,2,1,1,Dynamite,199,80642441,435982
1,TOMORROW X TOGETHER,The Chaos Chapter: FREEZE,24279001,166105,"11 Nov 2021, 13:17",https://lastfm.freetls.fastly.net/i/u/174s/d5e...,8,2,1,ANTI-ROMANTIC,215,3867105,136965
2,j-hope,Hope World,23430617,195267,"09 Sep 2021, 19:49",https://lastfm.freetls.fastly.net/i/u/174s/57b...,7,3,1,Hope World,204,3106115,118107
3,BTS,DARK & WILD,16051931,200195,"10 Oct 2021, 21:08",https://lastfm.freetls.fastly.net/i/u/174s/67f...,14,4,1,Intro: What Am I to You,165,776556,66698
4,Lisa,LALISA,10477044,230773,"09 Sep 2021, 18:54",https://lastfm.freetls.fastly.net/i/u/174s/386...,2,5,1,LALISA,200,5342514,138270
...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,Stray Kids,NOEASY,9516958,93995,"12 Dec 2021, 21:38",https://lastfm.freetls.fastly.net/i/u/174s/8f3...,14,8,13,WOLFGANG,192.0,962894,40779
110,NCT,NCT 2018 EMPATHY,6431069,149895,"03 Mar 2018, 02:22",https://lastfm.freetls.fastly.net/i/u/174s/a84...,14,15,13,OUTRO: VISION,57.0,95244,13050
111,BTS,DARK & WILD,16051931,200195,"10 Oct 2021, 21:08",https://lastfm.freetls.fastly.net/i/u/174s/67f...,14,4,14,Outro : Do You Think It Makes Sense?,172.0,29134,3661
112,Stray Kids,NOEASY,9516958,93995,"12 Dec 2021, 21:38",https://lastfm.freetls.fastly.net/i/u/174s/8f3...,14,8,14,Mixtape : OH,212.0,1092738,51049


### 5) Final adjustments
The top album data is almost ready. Now it is time to do some final adjustments to data types and make it ready to load into Tableau.

In [21]:
# Let's check the data
albuminfo_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   artist_x        114 non-null    object
 1   name_x          114 non-null    object
 2   playcount_x     114 non-null    object
 3   listeners_x     114 non-null    object
 4   published       114 non-null    object
 5   jpg             114 non-null    object
 6   numtracks       114 non-null    int64 
 7   rank            114 non-null    object
 8   trackno         114 non-null    object
 9   trackname_melt  114 non-null    object
 10  tracksdur_melt  114 non-null    object
 11  playcount_y     114 non-null    object
 12  listeners_y     114 non-null    object
dtypes: int64(1), object(12)
memory usage: 11.7+ KB


In [22]:
# transform rank into integer
albuminfo_data["rank"] = albuminfo_data["rank"].copy().astype("int64")

# transform tracksdur_melt into integer
albuminfo_data["tracksdur_melt"] = albuminfo_data["tracksdur_melt"].copy().astype("int64")

# transform playcounts into integer
albuminfo_data["playcount_x"] = albuminfo_data["playcount_x"].copy().astype("int64")
albuminfo_data["listeners_x"] = albuminfo_data["listeners_x"].copy().astype("int64")
albuminfo_data["playcount_y"] = albuminfo_data["playcount_y"].copy().astype("int64")
albuminfo_data["listeners_y"] = albuminfo_data["listeners_y"].copy().astype("int64")

# transform published into datatime
albuminfo_data["published"] = pd.to_datetime(albuminfo_data["published"], errors="coerce")
albuminfo_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   artist_x        114 non-null    object        
 1   name_x          114 non-null    object        
 2   playcount_x     114 non-null    int64         
 3   listeners_x     114 non-null    int64         
 4   published       113 non-null    datetime64[ns]
 5   jpg             114 non-null    object        
 6   numtracks       114 non-null    int64         
 7   rank            114 non-null    int64         
 8   trackno         114 non-null    object        
 9   trackname_melt  114 non-null    object        
 10  tracksdur_melt  114 non-null    int64         
 11  playcount_y     114 non-null    int64         
 12  listeners_y     114 non-null    int64         
dtypes: datetime64[ns](1), int64(7), object(5)
memory usage: 11.7+ KB


In [23]:
# rename some columns
albuminfo_data = albuminfo_data.rename(columns={
    "artist_x" : "artist",
    "name_x" : "album",
    "playcount_x" : "alb_playcount",
    "listeners_x" : "alb_listeners",
    "published" : "alb_published",
    "rank" : "alb_rank",
    "trackname_melt" : "trackname",
    "tracksdur_melt" : "trackduration",
    "playcount_y" : "trc_playcount",
    "listeners_y" : "trc_listeners",
    "rank" : "alb_rank"})

#rearrange column order
albuminfo_data = albuminfo_data[["artist", "album", "alb_published", "alb_rank", "alb_playcount", "alb_listeners", "jpg", "trackno", "trackname", "trackduration", "trc_playcount", "trc_listeners"]]

#edit artist column to be all uppercase (avoid unmatches)
albuminfo_data["artist"] = albuminfo_data["artist"].apply(lambda x: x.upper())


In [24]:
# Data is ready!
albuminfo_data

,artist,album,alb_published,alb_rank,alb_playcount,alb_listeners,jpg,trackno,trackname,trackduration,trc_playcount,trc_listeners
0,BTS,Dynamite,2021-11-11 19:26:00,1,28673011,248325,https://lastfm.freetls.fastly.net/i/u/174s/41b...,1,Dynamite,199,80642441,435982
1,TOMORROW X TOGETHER,The Chaos Chapter: FREEZE,2021-11-11 13:17:00,2,24279001,166105,https://lastfm.freetls.fastly.net/i/u/174s/d5e...,1,ANTI-ROMANTIC,215,3867105,136965
2,J-HOPE,Hope World,2021-09-09 19:49:00,3,23430617,195267,https://lastfm.freetls.fastly.net/i/u/174s/57b...,1,Hope World,204,3106115,118107
3,BTS,DARK & WILD,2021-10-10 21:08:00,4,16051931,200195,https://lastfm.freetls.fastly.net/i/u/174s/67f...,1,Intro: What Am I to You,165,776556,66698
4,LISA,LALISA,2021-09-09 18:54:00,5,10477044,230773,https://lastfm.freetls.fastly.net/i/u/174s/386...,1,LALISA,200,5342514,138270
...,...,...,...,...,...,...,...,...,...,...,...,...
109,STRAY KIDS,NOEASY,2021-12-12 21:38:00,8,9516958,93995,https://lastfm.freetls.fastly.net/i/u/174s/8f3...,13,WOLFGANG,192,962894,40779
110,NCT,NCT 2018 EMPATHY,2018-03-03 02:22:00,15,6431069,149895,https://lastfm.freetls.fastly.net/i/u/174s/a84...,13,OUTRO: VISION,57,95244,13050
111,BTS,DARK & WILD,2021-10-10 21:08:00,4,16051931,200195,https://lastfm.freetls.fastly.net/i/u/174s/67f...,14,Outro : Do You Think It Makes Sense?,172,29134,3661
112,STRAY KIDS,NOEASY,2021-12-12 21:38:00,8,9516958,93995,https://lastfm.freetls.fastly.net/i/u/174s/8f3...,14,Mixtape : OH,212,1092738,51049


### 6) End of processing top albums data
Now I have a nice dataset of the top 20 k-pop albums according to LastFM stats. Let's check "BLACKPINK".

In [25]:
albuminfo_data[albuminfo_data["artist"]=="BLACKPINK"]

,artist,album,alb_published,alb_rank,alb_playcount,alb_listeners,jpg,trackno,trackname,trackduration,trc_playcount,trc_listeners
6,BLACKPINK,SQUARE UP,2021-07-07 16:47:00,7,10644536,292527,https://lastfm.freetls.fastly.net/i/u/174s/f18...,1,뚜두뚜두 (DDU-DU DDU-DU),215,190195,8230
15,BLACKPINK,SQUARE ONE,2020-12-12 21:48:00,16,6143917,253888,https://lastfm.freetls.fastly.net/i/u/174s/fad...,1,WHISTLE,213,2897836,192409
19,BLACKPINK,SQUARE TWO,2020-12-12 21:49:00,20,5433989,211087,https://lastfm.freetls.fastly.net/i/u/174s/f91...,1,불장난,197,303907,7704
26,BLACKPINK,SQUARE UP,2021-07-07 16:47:00,7,10644536,292527,https://lastfm.freetls.fastly.net/i/u/174s/f18...,2,Forever Young,237,3240972,170302
32,BLACKPINK,SQUARE ONE,2020-12-12 21:48:00,16,6143917,253888,https://lastfm.freetls.fastly.net/i/u/174s/fad...,2,BOOMBAYAH,242,4024753,238646
35,BLACKPINK,SQUARE TWO,2020-12-12 21:49:00,20,5433989,211087,https://lastfm.freetls.fastly.net/i/u/174s/f91...,2,STAY,230,1893093,130463
40,BLACKPINK,SQUARE UP,2021-07-07 16:47:00,7,10644536,292527,https://lastfm.freetls.fastly.net/i/u/174s/f18...,3,REALLY,197,2017655,125318
48,BLACKPINK,SQUARE TWO,2020-12-12 21:49:00,20,5433989,211087,https://lastfm.freetls.fastly.net/i/u/174s/f91...,3,WHISTLE (Acoustic Ver.),212,113698,11788
53,BLACKPINK,SQUARE UP,2021-07-07 16:47:00,7,10644536,292527,https://lastfm.freetls.fastly.net/i/u/174s/f18...,4,SEE U LATER,216,1836408,119738
61,BLACKPINK,SQUARE TWO,2020-12-12 21:49:00,20,5433989,211087,https://lastfm.freetls.fastly.net/i/u/174s/f91...,4,휘파람,211,327743,9508


# Part B - Twitter API
### 1) Extract Twitter Account Data
Now I extract twitter data using tweepy, a wrapper built for Python. The goal of this part is to find the official twitter accounts of the top kpop performers and extract a couple of their latest tweets together with their relevant metrics such as number of likes and retweets. I use the Twitter API's `search_users` method which is similar to Find People button on Twitter.com. 

It is quite hard to programatically get the correct Twitter accounts for each artist because Twitter search will most likely return unrelevant results. For example, unoficial twitter accounts of fans or some other celebrity that happens to have the same name of the K-pop artist we are interested in.

Below, I cover three attemps I did to get the best data possible with minimal manual edits. 

In [26]:
# Twitter API
import tweepy

# bearer_token="AAAAAAAAAAAAAAAAAAAAANIWWgEAAAAALH7nhzTyg3Tcuul4stzcHfu00Xg%3DtSAQBbebzHEIRaXjnlgzJrAEyopmtvYnj0ri5NFYVj0OpWfJ24"
consumer_key="PaT5BsKCbJXgPepUc18DXik5S"
consumer_secret="eomtEOjuU295AD8qSNgK4fF01XE1SK8KTYMjs27n0h8WJSVXe0"
# access_token="2803650524"
# access_token_secret="v3dQg2CcLgpsRgMKd3N2UUDIHWcMcqjgPQC1G3aXizJyc"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
API = tweepy.API(auth=auth)

### Attempt 1
I use the dataframe containing top albums and use Twitter's search engine to find verified users. I add the word "official" to the artist name to increase chances of getting the official twitter account of the k-pop artist. For example: "BTS official".

In [27]:
iden = []
artist = []
name = []
screen_name = []
followers_count = []
verified = []

for a in top_albums["artist"].unique():
    user = API.search_users(q=a+" official")
    ## while loop to find the official twitter account of musician 
    i = 0
    while (i < len(user)-1) and (user[i]._json["verified"]==False):
        i+=1
    else: 
        artist.append(a)
        iden.append(user[i]._json["id"]), 
        name.append(user[i]._json["name"]), 
        screen_name.append(user[i]._json["screen_name"]), 
        followers_count.append(user[i]._json["followers_count"]), 
        verified.append(user[i]._json["verified"])
    print("end of {}".format(a))

end of BTS
end of TOMORROW X TOGETHER
end of J-HOPE
end of LISA
end of BLACKPINK
end of STRAY KIDS
end of LOONA
end of ROSÉ
end of ENHYPEN
end of NCT
end of ITZY
end of IZ*ONE
end of K/DA


In [28]:
twitter_acct = dict(id=iden,artist=artist,name=name,screen_name=screen_name,followers_count=followers_count)
twitter_acct_df = pd.DataFrame(twitter_acct)

# Let's check results
twitter_acct_df

,id,artist,name,screen_name,followers_count
0,1409798257,BTS,BTS_official,bts_bighit,36367258
1,1059325677202157568,TOMORROW X TOGETHER,TXT JAPAN OFFICIAL,TXT_bighit_jp,2209608
2,271444539,J-HOPE,Kaplan Foundation,KaplanFnd,732
3,18061669,LISA,Sen. Lisa Murkowski,lisamurkowski,325279
4,878204370403250176,BLACKPINK,BLACKPINK GLOBAL BLINK,ygofficialblink,7805453
5,1103477383158411264,STRAY KIDS,Stray Kids Japan Official,Stray_Kids_JP,1372878
6,806918031964192769,LOONA,Loona Acts Concepts,official_loona,23
7,3315658770,ROSÉ,Rose Marie-Official,RoseMarie4Real,92001
8,1293890751848275968,ENHYPEN,ENHYPEN OFFICIAL,ENHYPEN,4177566
9,892285827195084800,NCT,NCT_OFFICIAL_JP,NCT_OFFICIAL_JP,2965803


Looking at the results above (results as of 2021/12), we can see that for some of the artists, I was not able to get the correct official Twitter account. For example, Twitter API returned @KaplanFnd (U.S. based charity foundation) for "J-Hope" and @lisamurkowski (U.S. senator) for Lisa. Both J-Hope and Lisa does not seem to have an official Twitter account.

Other accounts that are incorrect are @TXT_bighit_jp (it is an official account for Japanese audience only), @official_loona (only 23 followers and not official) and @RoseMarie4Real (American actress).

### Attempt 2
Below, I try a different approach to get better results. I use the API to search Twitter for `artist name  + official`, then I store all accounts results into a dataframe. I sort and filter this dataframe to display only accounts with the highest number of followers for each artist.  

In [29]:
user = []
for a in top_albums["artist"].unique():
    user.append(API.search_users(q=a+" official"))

In [30]:
user = list(zip(top_albums["artist"].unique(), user))

In [31]:
id = []
artist = []
name = []
screen_name = []
followers_count = []
verified = []
profpic = []

for a, t in user:
        for k in t:
            artist.append(a)
            id.append(k._json["id"])
            name.append(k._json["name"])
            screen_name.append(k._json["screen_name"])
            followers_count.append(k._json["followers_count"])
            verified.append(k._json["verified"])
            profpic.append(k._json["profile_image_url"])

twitter_acct2 = pd.DataFrame(dict(artist=artist, id=id, name=name,screen_name=screen_name,followers_count=followers_count, verified=verified, profpic=profpic))
twitter_acct_df2 = twitter_acct2[twitter_acct2.groupby(["artist"])["followers_count"].transform(max) == twitter_acct2['followers_count']].reset_index(drop=True)

# Check results
twitter_acct_df2

,artist,id,name,screen_name,followers_count,verified,profpic
0,BTS,1409798257,BTS_official,bts_bighit,36366674,True,http://pbs.twimg.com/profile_images/1387421728...
1,TOMORROW X TOGETHER,1083198663424237569,TXT OFFICIAL,TXT_bighit,8612577,True,http://pbs.twimg.com/profile_images/1416776136...
2,J-HOPE,812023764477157376,J-Hope Py Official ~ HOPE WORLD☀️🐿💚,JHope_Paraguay,16749,False,http://pbs.twimg.com/profile_images/1457746134...
3,LISA,18061669,Sen. Lisa Murkowski,lisamurkowski,325279,True,http://pbs.twimg.com/profile_images/1061074014...
4,BLACKPINK,408177579,YG FAMILY,ygent_official,8230084,True,http://pbs.twimg.com/profile_images/8694461102...
5,STRAY KIDS,859297966581891072,Stray Kids,Stray_Kids,6876705,True,http://pbs.twimg.com/profile_images/1458449377...
6,LOONA,1407603879026462722,loonatheworld_jp_official,loonaJPofficial,80205,False,http://pbs.twimg.com/profile_images/1407682642...
7,ROSÉ,53892481,Hampshire Cricket,hantscricket,96794,True,http://pbs.twimg.com/profile_images/1439838811...
8,ENHYPEN,1293890751848275968,ENHYPEN OFFICIAL,ENHYPEN,4177702,True,http://pbs.twimg.com/profile_images/1469110346...
9,NCT,892285827195084800,NCT_OFFICIAL_JP,NCT_OFFICIAL_JP,2965739,True,http://pbs.twimg.com/profile_images/8994818182...


Results seem to have got better (results as od 2021/12), but are still not perfect. 

- **Spot on**: @TXT_bighit and @Stray_Kids are now correct
- **Got better**: Loona now shows a Japanese language unverified group account (seems to be a legit account, though)
- **Still off**: NCT (not the global account), J-Hope, and Rose. 
- **Now it is off**: Blackpink now shows the official account of the management company not of the group itself. 

It will be very difficult to find the perfect match of twitter accounts I want by only using twitter's search engine. After a quick web search, I found out that Loona does have its own official twitter account but because its name is stylized (LOOΠΔ), I wasn't able to properly find it. Also, there is no guarantee I will be able to get results as good as these once new artists with whacky names appear in our top album list. 

### Attempt 3
A better approach would be to scrape this Kpop database (curated by fans), locate the artists that I want and extract their twitter accounts: https://kpopping.com/profiles/the-groups
<br>Let's quickly scrape twitter accounts using this method

In [32]:
from bs4 import BeautifulSoup
import requests

# Webscraping
site = "https://kpopping.com/profiles/the-groups"
resp = requests.get(site, timeout=5)
soup = BeautifulSoup(resp.content, "html.parser")

In [33]:
url = []
band = []

# Store data into dataframe
for item in soup.findAll("a", href=True):
    if item["href"][:16] == "/profiles/group/":
        url.append(item["href"])
        band.append(item.text)

kpopdb_scraped = pd.DataFrame(dict(url=url, band=band))
kpopdb_scraped.head(10)

,url,band
0,/profiles/group/submission/skip-SW,submit new
1,/profiles/group/CHIC-IDLE,CHIC&IDLE
2,/profiles/group/Red-Velvet-IRENE-SEULGI,Red Velvet - IRENE & SEULGI
3,/profiles/group/G-I-DLE,(G)I-DLE
4,/profiles/group/05Class,05Class
5,/profiles/group/100,100%
6,/profiles/group/10cm,10cm
7,/profiles/group/10X10,10X10
8,/profiles/group/14U,14U
9,/profiles/group/15,15&


In [34]:
# Filter results with top artists
kpopdb_scraped["band"] = kpopdb_scraped["band"].apply(lambda x: x.upper())

twitter_acct_df_temp = twitter_acct_df2.copy()
twitter_acct_df_temp["artist2"] = twitter_acct_df_temp["artist"].apply(lambda x: x.upper())

kpopdb_scraped_top = kpopdb_scraped[kpopdb_scraped["band"].isin(twitter_acct_df_temp["artist2"])].copy()

In [35]:
accts = []
for idx in range(kpopdb_scraped_top.shape[0]):
    site = "https://kpopping.com"+kpopdb_scraped_top.iloc[idx,:]["url"]
    resp = requests.get(site, timeout=5)
    soup = BeautifulSoup(resp.content, "html.parser")
    for acc in soup.findAll("a", class_="fab fa-twitter", href=True):
        accts.append(acc["href"]) 
kpopdb_scraped_top["accts"] = accts

#check results
kpopdb_scraped_top

,url,band,accts
159,/profiles/group/BLACKPINK,BLACKPINK,https://twitter.com/blackpink
188,/profiles/group/BTS,BTS,https://twitter.com/BTS_twt
295,/profiles/group/ENHYPEN,ENHYPEN,https://twitter.com/ENHYPEN
456,/profiles/group/ITZY,ITZY,https://twitter.com/ITZYOfficial
459,/profiles/group/IZ-ONE,IZ*ONE,https://twitter.com/official_izone
529,/profiles/group/LOONA,LOONA,https://twitter.com/loonatheworld?lang=en
614,/profiles/group/NCT,NCT,https://twitter.com/nctsmtown
799,/profiles/group/Stray-Kids,STRAY KIDS,https://twitter.com/Stray_Kids?s=09


We can see above that the kpop database does not have all artists I get from LastFM. For example, K/DA does not appear because it does not exist in the database. I also fail TOMORROW X TOGETHER because apparently this group is better known by its abbreviated name - "TXT" - and this is how it is written in the databse. 

However, for all other artists, it seems that I succeed in getting correct official twitter accounts. Let's now use the Twitter API to get those data and build our own database by combining results from Attempr 2 and the scraped results.  

To do that I will need to first edit the `accts` column and get screen_names.

In [36]:
kpopdb_scraped_top["screen_name"] = kpopdb_scraped_top["accts"].apply(lambda x: x.split("/")[-1])
kpopdb_scraped_top["screen_name"] = kpopdb_scraped_top["screen_name"].apply(lambda x: x.split("?")[0])
kpopdb_scraped_top

,url,band,accts,screen_name
159,/profiles/group/BLACKPINK,BLACKPINK,https://twitter.com/blackpink,blackpink
188,/profiles/group/BTS,BTS,https://twitter.com/BTS_twt,BTS_twt
295,/profiles/group/ENHYPEN,ENHYPEN,https://twitter.com/ENHYPEN,ENHYPEN
456,/profiles/group/ITZY,ITZY,https://twitter.com/ITZYOfficial,ITZYOfficial
459,/profiles/group/IZ-ONE,IZ*ONE,https://twitter.com/official_izone,official_izone
529,/profiles/group/LOONA,LOONA,https://twitter.com/loonatheworld?lang=en,loonatheworld
614,/profiles/group/NCT,NCT,https://twitter.com/nctsmtown,nctsmtown
799,/profiles/group/Stray-Kids,STRAY KIDS,https://twitter.com/Stray_Kids?s=09,Stray_Kids


Now I use Twitter API to extract user information. Same procedure I used above, so I will skip all commentary.

In [37]:
users = []
for sn in range(kpopdb_scraped_top.shape[0]):
    user = API.get_user(screen_name=kpopdb_scraped_top.iloc[sn,:]["screen_name"])
    users.append(user)

In [38]:
band_users = list(zip(kpopdb_scraped_top["band"], users))

In [39]:
id = []
artist = []
name = []
screen_name = []
followers_count = []
verified = []
profpic = []

for a, k in band_users:
        artist.append(a)
        id.append(k._json["id"])
        name.append(k._json["name"])
        screen_name.append(k._json["screen_name"])
        followers_count.append(k._json["followers_count"])
        verified.append(k._json["verified"])
        profpic.append(k._json["profile_image_url"])

twitter_acct_df3 = pd.DataFrame(dict(artist=artist, id=id, name=name,screen_name=screen_name,followers_count=followers_count, verified=verified, profpic=profpic))

# check results
twitter_acct_df3

,artist,id,name,screen_name,followers_count,verified,profpic
0,BLACKPINK,1273850774380883975,BLACKPINKOFFICIAL,BLACKPINK,5626296,True,http://pbs.twimg.com/profile_images/1311878593...
1,BTS,335141638,방탄소년단,BTS_twt,42725656,True,http://pbs.twimg.com/profile_images/1437401193...
2,ENHYPEN,1293890751848275968,ENHYPEN OFFICIAL,ENHYPEN,4177569,True,http://pbs.twimg.com/profile_images/1469110346...
3,ITZY,1062578718214770688,ITZY,ITZYofficial,4251088,True,http://pbs.twimg.com/profile_images/1468958695...
4,IZ*ONE,1034744720537219073,official_IZONE,official_izone,1329678,True,http://pbs.twimg.com/profile_images/1335600379...
5,LOONA,770959712791597056,이달의 소녀(LOOΠΔ),loonatheworld,840912,True,http://pbs.twimg.com/profile_images/7709919458...
6,NCT,4811011050,NCT,NCTsmtown,8064592,True,http://pbs.twimg.com/profile_images/1469230267...
7,STRAY KIDS,859297966581891072,Stray Kids,Stray_Kids,6876709,True,http://pbs.twimg.com/profile_images/1458449377...


Now I have two good tables containing twitter accounts of K-pop artists (Attemp 2 and Attemp 3). The goal now is to grab the correct twitter accounts from each table.
- Attempt 3) Scraped data has correct twitter accounts but is incomplete.
- Attempt 2) Data extracted using Twitter API search method is complete, but some entries are incorrect.

Below, I build a "complete" dataset by concatenating tables from both attempts, then I remove all duplicates from the "Twitter API" data. Next, I remove the data for accounts that are obviously of non-K-pop stars (Lisa and Rose).  

In [40]:
#build "order" column so that I can tell Python the exact duplicates to remove (keep the "1s")
twitter_acct_df3["order"]=1
twitter_acct_df3

,artist,id,name,screen_name,followers_count,verified,profpic,order
0,BLACKPINK,1273850774380883975,BLACKPINKOFFICIAL,BLACKPINK,5626296,True,http://pbs.twimg.com/profile_images/1311878593...,1
1,BTS,335141638,방탄소년단,BTS_twt,42725656,True,http://pbs.twimg.com/profile_images/1437401193...,1
2,ENHYPEN,1293890751848275968,ENHYPEN OFFICIAL,ENHYPEN,4177569,True,http://pbs.twimg.com/profile_images/1469110346...,1
3,ITZY,1062578718214770688,ITZY,ITZYofficial,4251088,True,http://pbs.twimg.com/profile_images/1468958695...,1
4,IZ*ONE,1034744720537219073,official_IZONE,official_izone,1329678,True,http://pbs.twimg.com/profile_images/1335600379...,1
5,LOONA,770959712791597056,이달의 소녀(LOOΠΔ),loonatheworld,840912,True,http://pbs.twimg.com/profile_images/7709919458...,1
6,NCT,4811011050,NCT,NCTsmtown,8064592,True,http://pbs.twimg.com/profile_images/1469230267...,1
7,STRAY KIDS,859297966581891072,Stray Kids,Stray_Kids,6876709,True,http://pbs.twimg.com/profile_images/1458449377...,1


In [41]:
#build "order" column so that I can tell Python the exact duplicates to remove (keep the "1s")
twitter_acct_df2["order"]=2
twitter_acct_df2

,artist,id,name,screen_name,followers_count,verified,profpic,order
0,BTS,1409798257,BTS_official,bts_bighit,36366674,True,http://pbs.twimg.com/profile_images/1387421728...,2
1,TOMORROW X TOGETHER,1083198663424237569,TXT OFFICIAL,TXT_bighit,8612577,True,http://pbs.twimg.com/profile_images/1416776136...,2
2,J-HOPE,812023764477157376,J-Hope Py Official ~ HOPE WORLD☀️🐿💚,JHope_Paraguay,16749,False,http://pbs.twimg.com/profile_images/1457746134...,2
3,LISA,18061669,Sen. Lisa Murkowski,lisamurkowski,325279,True,http://pbs.twimg.com/profile_images/1061074014...,2
4,BLACKPINK,408177579,YG FAMILY,ygent_official,8230084,True,http://pbs.twimg.com/profile_images/8694461102...,2
5,STRAY KIDS,859297966581891072,Stray Kids,Stray_Kids,6876705,True,http://pbs.twimg.com/profile_images/1458449377...,2
6,LOONA,1407603879026462722,loonatheworld_jp_official,loonaJPofficial,80205,False,http://pbs.twimg.com/profile_images/1407682642...,2
7,ROSÉ,53892481,Hampshire Cricket,hantscricket,96794,True,http://pbs.twimg.com/profile_images/1439838811...,2
8,ENHYPEN,1293890751848275968,ENHYPEN OFFICIAL,ENHYPEN,4177702,True,http://pbs.twimg.com/profile_images/1469110346...,2
9,NCT,892285827195084800,NCT_OFFICIAL_JP,NCT_OFFICIAL_JP,2965739,True,http://pbs.twimg.com/profile_images/8994818182...,2


In [42]:
# keep only verified accounts, then concatenate tables.
twitter_acct_df4 = pd.concat([twitter_acct_df2, twitter_acct_df3], axis=0)

# drop duplicates by keeping entries from table 1 (scraped data)
twitter_acct_df5 = twitter_acct_df4[twitter_acct_df4["verified"]].sort_values(["artist", "order"]).drop_duplicates("artist", keep="first")
twitter_acct_df5

,artist,id,name,screen_name,followers_count,verified,profpic,order
0,BLACKPINK,1273850774380883975,BLACKPINKOFFICIAL,BLACKPINK,5626296,True,http://pbs.twimg.com/profile_images/1311878593...,1
1,BTS,335141638,방탄소년단,BTS_twt,42725656,True,http://pbs.twimg.com/profile_images/1437401193...,1
2,ENHYPEN,1293890751848275968,ENHYPEN OFFICIAL,ENHYPEN,4177569,True,http://pbs.twimg.com/profile_images/1469110346...,1
3,ITZY,1062578718214770688,ITZY,ITZYofficial,4251088,True,http://pbs.twimg.com/profile_images/1468958695...,1
4,IZ*ONE,1034744720537219073,official_IZONE,official_izone,1329678,True,http://pbs.twimg.com/profile_images/1335600379...,1
12,K/DA,1289225491233284097,K/DA,KDA_MUSIC,440170,True,http://pbs.twimg.com/profile_images/1321436562...,2
3,LISA,18061669,Sen. Lisa Murkowski,lisamurkowski,325279,True,http://pbs.twimg.com/profile_images/1061074014...,2
5,LOONA,770959712791597056,이달의 소녀(LOOΠΔ),loonatheworld,840912,True,http://pbs.twimg.com/profile_images/7709919458...,1
6,NCT,4811011050,NCT,NCTsmtown,8064592,True,http://pbs.twimg.com/profile_images/1469230267...,1
7,ROSÉ,53892481,Hampshire Cricket,hantscricket,96794,True,http://pbs.twimg.com/profile_images/1439838811...,2


In [43]:
#id that are not kpop stars (remove from table)
notkpopstar = [53892481, 18061669]

`notkpopstar` is a list of errors that I use as a reference to remove wrong entries from my dataset. In the future, when I come back to this script to refresh my dashboard, I will need to eyeball `twitter_acct_df5` table and keep adding the ids of wrong Twitter accounts that were newly added to the dataset. 

There must be a smarter way to do this, but I will just leave the way it since it is not a lot of work to add id numbers to the list.

In [44]:
twitter_acct_df5 = twitter_acct_df5[~twitter_acct_df5["id"].isin(notkpopstar)].reset_index(drop=True)

# check results
twitter_acct_df5

,artist,id,name,screen_name,followers_count,verified,profpic,order
0,BLACKPINK,1273850774380883975,BLACKPINKOFFICIAL,BLACKPINK,5626296,True,http://pbs.twimg.com/profile_images/1311878593...,1
1,BTS,335141638,방탄소년단,BTS_twt,42725656,True,http://pbs.twimg.com/profile_images/1437401193...,1
2,ENHYPEN,1293890751848275968,ENHYPEN OFFICIAL,ENHYPEN,4177569,True,http://pbs.twimg.com/profile_images/1469110346...,1
3,ITZY,1062578718214770688,ITZY,ITZYofficial,4251088,True,http://pbs.twimg.com/profile_images/1468958695...,1
4,IZ*ONE,1034744720537219073,official_IZONE,official_izone,1329678,True,http://pbs.twimg.com/profile_images/1335600379...,1
5,K/DA,1289225491233284097,K/DA,KDA_MUSIC,440170,True,http://pbs.twimg.com/profile_images/1321436562...,2
6,LOONA,770959712791597056,이달의 소녀(LOOΠΔ),loonatheworld,840912,True,http://pbs.twimg.com/profile_images/7709919458...,1
7,NCT,4811011050,NCT,NCTsmtown,8064592,True,http://pbs.twimg.com/profile_images/1469230267...,1
8,STRAY KIDS,859297966581891072,Stray Kids,Stray_Kids,6876709,True,http://pbs.twimg.com/profile_images/1458449377...,1
9,TOMORROW X TOGETHER,1083198663424237569,TXT OFFICIAL,TXT_bighit,8612577,True,http://pbs.twimg.com/profile_images/1416776136...,2


Before I proceed to the next step, I will save the profile pictures so that I can use those in the dashboard.

In [53]:
# loop through every profile picture and save it
# I use generic file names so that it is easy to update
for idx, url in enumerate(twitter_acct_df5["profpic"]):
    filename = twitter_acct_df5.loc[idx,"screen_name"]+"_pic"+url[-4:]
    urllib.request.urlretrieve(url, filename)
    print("saved "+filename)

saved BLACKPINK_pic.jpg
saved BTS_twt_pic.jpg
saved ENHYPEN_pic.jpg
saved ITZYofficial_pic.jpg
saved official_izone_pic.jpg
saved KDA_MUSIC_pic.jpg
saved loonatheworld_pic.jpg
saved NCTsmtown_pic.jpg
saved Stray_Kids_pic.jpg
saved TXT_bighit_pic.jpg


### 2) Extract Twitter Timelines
The next step is to extract the timelines of the twitter accounts found above using the `API.user_timeline` method. Twitter API will return the latest 20 tweets from each acount (I set the API to exclude replies and retweets, so for some accounts it might be less than 20 results)

In [46]:
t_artist = []
t_text = []
t_date = []
t_rtcount = []
t_fvcount = []

for k,i in enumerate(twitter_acct_df5["id"]):
    print(twitter_acct_df5["artist"][k])
    tweets = API.user_timeline(user_id=i, exclude_replies=True, include_rts=False)
    for t in range(len(tweets)):
        t_artist.append(twitter_acct_df5["artist"][k])
        t_text.append(tweets[t]._json["text"])
        t_date.append(tweets[t]._json["created_at"])
        t_rtcount.append(tweets[t]._json["retweet_count"])
        t_fvcount.append(tweets[t]._json["favorite_count"])        

BLACKPINK
BTS
ENHYPEN
ITZY
IZ*ONE
K/DA
LOONA
NCT
STRAY KIDS
TOMORROW X TOGETHER


In [47]:
tweet = dict(t_artist=t_artist, t_text=t_text, t_date=t_date, t_rtcount=t_rtcount, t_fvcount=t_fvcount)
tweet_df = pd.DataFrame(tweet)

Chck results for twitter timelines. This is the results for BLACKPINK.

In [48]:
tweet_df[tweet_df["t_artist"]=="BLACKPINK"]

,t_artist,t_text,t_date,t_rtcount,t_fvcount
0,BLACKPINK,👉Hit you with that #DDUDUDDUDU👉\nItems inspire...,Tue Dec 21 08:01:17 +0000 2021,8048,57259
1,BLACKPINK,#ESCAPE2021 IN YOUR AREA https://t.co/Dbx8tiDw...,Thu Dec 16 04:30:24 +0000 2021,19465,107397
2,BLACKPINK,#BLACKPINK ‘Forever Young’ DANCE PRACTICE VIDE...,Mon Dec 13 01:37:25 +0000 2021,20562,94224
3,BLACKPINK,#BLACKPINK ‘STAY’ M/V HITS 300 MILLION VIEWS @...,Sat Dec 11 23:01:54 +0000 2021,30603,123454
4,BLACKPINK,Thank you so much for streaming #LALISA and #M...,Thu Dec 09 08:44:19 +0000 2021,24778,91178
5,BLACKPINK,Thank you so much for streaming #OnTheGround a...,Thu Dec 09 08:43:57 +0000 2021,15895,65162
6,BLACKPINK,#BLACKPINK's #THEALBUM is now in #DolbyAtmos 🎶...,Wed Dec 08 09:06:36 +0000 2021,9623,53878
7,BLACKPINK,#LISA -LALISA- PHOTOBOOK [SPECIAL EDITION]\nPr...,Wed Dec 08 02:01:43 +0000 2021,18559,82517
8,BLACKPINK,#LISA FIRST SINGLE VINYL LP LALISA [LIMITED ED...,Tue Nov 30 03:02:33 +0000 2021,20841,106785
9,BLACKPINK,#LISA ‘MONEY’ EXCLUSIVE PERFORMANCE VIDEO HITS...,Sun Nov 28 23:01:08 +0000 2021,38312,133118


### 3) Merge Twitter datasets (and some small adjustments to the data) 
Now I merge the dataset containing twitter accounts with twitter timelines to get a single twitter dataset.

In [49]:
twitter_data = twitter_acct_df5.merge(right=tweet_df, left_on="artist", right_on="t_artist")
twitter_data

,artist,id,name,screen_name,followers_count,verified,profpic,order,t_artist,t_text,t_date,t_rtcount,t_fvcount
0,BLACKPINK,1273850774380883975,BLACKPINKOFFICIAL,BLACKPINK,5626296,True,http://pbs.twimg.com/profile_images/1311878593...,1,BLACKPINK,👉Hit you with that #DDUDUDDUDU👉\nItems inspire...,Tue Dec 21 08:01:17 +0000 2021,8048,57259
1,BLACKPINK,1273850774380883975,BLACKPINKOFFICIAL,BLACKPINK,5626296,True,http://pbs.twimg.com/profile_images/1311878593...,1,BLACKPINK,#ESCAPE2021 IN YOUR AREA https://t.co/Dbx8tiDw...,Thu Dec 16 04:30:24 +0000 2021,19465,107397
2,BLACKPINK,1273850774380883975,BLACKPINKOFFICIAL,BLACKPINK,5626296,True,http://pbs.twimg.com/profile_images/1311878593...,1,BLACKPINK,#BLACKPINK ‘Forever Young’ DANCE PRACTICE VIDE...,Mon Dec 13 01:37:25 +0000 2021,20562,94224
3,BLACKPINK,1273850774380883975,BLACKPINKOFFICIAL,BLACKPINK,5626296,True,http://pbs.twimg.com/profile_images/1311878593...,1,BLACKPINK,#BLACKPINK ‘STAY’ M/V HITS 300 MILLION VIEWS @...,Sat Dec 11 23:01:54 +0000 2021,30603,123454
4,BLACKPINK,1273850774380883975,BLACKPINKOFFICIAL,BLACKPINK,5626296,True,http://pbs.twimg.com/profile_images/1311878593...,1,BLACKPINK,Thank you so much for streaming #LALISA and #M...,Thu Dec 09 08:44:19 +0000 2021,24778,91178
...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,TOMORROW X TOGETHER,1083198663424237569,TXT OFFICIAL,TXT_bighit,8612577,True,http://pbs.twimg.com/profile_images/1416776136...,2,TOMORROW X TOGETHER,TXT (투모로우바이투게더) 'Sweet Dreams'\n(https://t.co/...,Wed Dec 22 09:00:18 +0000 2021,27020,87918
134,TOMORROW X TOGETHER,1083198663424237569,TXT OFFICIAL,TXT_bighit,8612577,True,http://pbs.twimg.com/profile_images/1416776136...,2,TOMORROW X TOGETHER,#투모로우바이투게더 가 #MOA 에게 선물하는 첫 번째 캐럴🎁\n'Sweet Dre...,Wed Dec 22 06:00:04 +0000 2021,26945,102016
135,TOMORROW X TOGETHER,1083198663424237569,TXT OFFICIAL,TXT_bighit,8612577,True,http://pbs.twimg.com/profile_images/1416776136...,2,TOMORROW X TOGETHER,[#TTIME] #휴닝카이 에게 주어진 2번의 유리 깨기 기회! 멋지게 성공할 수 ...,Wed Dec 22 05:00:29 +0000 2021,8974,55678
136,TOMORROW X TOGETHER,1083198663424237569,TXT OFFICIAL,TXT_bighit,8612577,True,http://pbs.twimg.com/profile_images/1416776136...,2,TOMORROW X TOGETHER,TO DO X TXT - EP.63 해피 홀리데이 파티 1편(Happy Holida...,Mon Dec 20 12:00:06 +0000 2021,11405,66069


Change the datatype for date

In [50]:
twitter_data["t_date"] = twitter_data["t_date"].apply(lambda x : pd.to_datetime(x, errors="coerce").date())
twitter_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 138 entries, 0 to 137
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   artist           138 non-null    object
 1   id               138 non-null    int64 
 2   name             138 non-null    object
 3   screen_name      138 non-null    object
 4   followers_count  138 non-null    int64 
 5   verified         138 non-null    bool  
 6   profpic          138 non-null    object
 7   order            138 non-null    int64 
 8   t_artist         138 non-null    object
 9   t_text           138 non-null    object
 10  t_date           138 non-null    object
 11  t_rtcount        138 non-null    int64 
 12  t_fvcount        138 non-null    int64 
dtypes: bool(1), int64(5), object(7)
memory usage: 14.2+ KB


In [51]:
# In Tableau, I will need to match album data and Twitter data using artist name. I set all names in uppercase to make sure all artists will match.
twitter_data["artist"] = twitter_data["artist"].apply(lambda x: x.upper())

# Twitter dataset is ready! 
twitter_data.to_excel("twitter_data.xlsx", sheet_name="Sheet1", index=False)
twitter_data["artist"].unique()

array(['BLACKPINK', 'BTS', 'ENHYPEN', 'ITZY', 'IZ*ONE', 'K/DA', 'LOONA',
       'NCT', 'STRAY KIDS', 'TOMORROW X TOGETHER'], dtype=object)

In [52]:
# In Tableau, I will need to match album data and Twitter data using artist name. I set all names in uppercase to make sure all artists will match.
albuminfo_data["artist"] = albuminfo_data["artist"].apply(lambda x: x.upper())

# Album dataset is ready! 
albuminfo_data.to_csv("albuminfo_data.csv", index=False)
albuminfo_data["artist"].unique()

array(['BTS', 'TOMORROW X TOGETHER', 'J-HOPE', 'LISA', 'BLACKPINK',
       'STRAY KIDS', 'LOONA', 'ROSÉ', 'ENHYPEN', 'NCT', 'ITZY', 'IZ*ONE',
       'K/DA'], dtype=object)

# Part 3) Visualize
The last step is to visualize the data in Tableau. See below a screenshot of it. The "K-Pop Monitor" (as I call it) has basic visualizations of top K-pop albums according to LastFM. The user needs to click one an album/artist to refresh the other charts, including the Twitter Analysis part at the bottom of the dashboard. 

This is not a particularly complex dashboard and it does not show any ground breaking analysis, but it is a good way to quickly grasp the K-pop industry in a "data-driven" way. The data grabbing process was streamlined so that I can update the dashboard with the latest tweets and album / track data by simply rerunning this Python notebook. The problem would be to update the pictures as I needed to manually setup the pictures inside Tableau. 

Check the dashboard at my [Tableau Public Account](https://public.tableau.com/views/projectk_16396618958480/Dashboard1?:language=en-GB&publish=yes&:display_count=n&:origin=viz_share_link). 

![](dashboard_ss.jpg)

This is the end of project K! Thanks for reading!